In [1]:
from torch_uncertainty import cli_main, init_args
from torch_uncertainty.baselines.classification import ResNet
from torch_uncertainty.optimization_procedures import optim_cifar10_resnet18
from torch_uncertainty.routines.classification import ClassificationSingle
from torch_uncertainty.datamodules import CIFAR10DataModule
from torch_uncertainty.losses import CrossEntropyLoss
from torchvision.datasets import CIFAR10
from torchvision import transforms
#from torch.nn import CrossEntropyLoss
from pytorch_lightning import Trainer
import torch

import os
import sys
from pathlib import Path
from cli_test_helpers import ArgvContext

In [2]:
torch.cuda.is_available()

True

### 1 - Models

In [3]:
baseline = ResNet(num_classes=10,
                loss=CrossEntropyLoss(),
                optimization_procedure=optim_cifar10_resnet18,
                version="std",
                in_channels=3, 
                arch=18)

mixup = ResNet(num_classes=10,
                loss=CrossEntropyLoss(),
                optimization_procedure=optim_cifar10_resnet18,
                version="std",
                in_channels=3, 
                arch=18, 
                mixup=True,
                mixup_alpha=0.2)

regmixup = ResNet(num_classes=10,
                loss=CrossEntropyLoss(),
                optimization_procedure=optim_cifar10_resnet18,
                version="std",
                in_channels=3,
                arch=18,
                reg_mixup=True,
                mixup_alpha=15)

### 2 - Data

In [4]:
transform = transforms.Compose([transforms.ToTensor(),
                                transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])
batch_size = 4

trainset = CIFAR10(root='./data', train=True, download=True, transform=transform)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=batch_size, shuffle=True, num_workers=2)

testset = CIFAR10(root='./data', train=False, download=True, transform=transform)
testloader = torch.utils.data.DataLoader(testset, batch_size=batch_size, shuffle=False, num_workers=2)

classes = ('plane', 'car', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck')

Files already downloaded and verified
Files already downloaded and verified


### 3. Training

In [5]:
root = Path(os.path.abspath(""))

# We mock the arguments for the trainer
with ArgvContext(
    "file.py",
    "--max_epochs",
    "1",
    "--enable_progress_bar",
    "False",
    "--num_estimators",
    "8",
    "--max_epochs",
    "2"
):
    args = init_args(network=ResNet, datamodule=CIFAR10DataModule, )

net_name = "logs/reset18-cifar10"

# datamodule
args.root = str(root / "data")
dm = CIFAR10DataModule(**vars(args))


In [7]:
Trainer(accelerator="gpu", devices=1)
results = cli_main(baseline, dm, root, net_name, args=args)

GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
GPU available: True (cuda), used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
d:\COURS\A5\Responsible AI\Code\regmixup_venv\Lib\site-packages\pytorch_lightning\trainer\setup.py:176: PossibleUserWarning: GPU available but not used. Set `accelerator` and `devices` using `Trainer(accelerator='gpu', devices=1)`.
  rank_zero_warn(


Files already downloaded and verified
Files already downloaded and verified



  | Name             | Type             | Params
------------------------------------------------------
0 | model            | _ResNet          | 11.2 M
1 | loss             | CrossEntropyLoss | 0     
2 | format_batch_fn  | Identity         | 0     
3 | val_cls_metrics  | MetricCollection | 0     
4 | test_cls_metrics | MetricCollection | 0     
5 | test_entropy_id  | Entropy          | 0     
------------------------------------------------------
11.2 M    Trainable params
0         Non-trainable params
11.2 M    Total params
44.727    Total estimated model params size (MB)


TypeError: CrossEntropyLoss.forward() missing 2 required positional arguments: 'input' and 'target'